# Jupyter Notebook for Calculating Statistics

**NOTE**: the `json` files that contain overall metrics and also the linear regression model results were created manually.
This can be done with some Python code very easily. However, the code for this is not included in this notebook.

In [111]:
import pandas as pd
import numpy as np
from math import sqrt
import altair as alt
from utils import save_dataset_to_csv
import statsmodels.formula.api as smf
from tqdm import tqdm
import tiktoken

In [112]:
model_metadata = {
    "alpacare-7B": {"model_type": "biomedical open", "model_size_in_b": 7},
    "biomedgpt7B": {"model_type": "biomedical open", "model_size_in_b": 7},
    "biomistral7B": {"model_type": "biomedical open", "model_size_in_b": 7},
    "claude_3.5-haiku": {"model_type": "generalist closed", "model_size_in_b": None},
    "claude_3.5-sonnet": {"model_type": "generalist closed", "model_size_in_b": None},
    "gemini_1.5_flash": {"model_type": "generalist closed", "model_size_in_b": None},
    "gemini_1.5_flash-8B": {"model_type": "generalist closed", "model_size_in_b": 8},
    "gpt4o": {"model_type": "generalist closed", "model_size_in_b": None},
    "gpt4o-mini": {"model_type": "generalist closed", "model_size_in_b": None},
    "gpt35": {"model_type": "generalist closed", "model_size_in_b": 175},
    "llama2_chat-7B": {"model_type": "generalist open", "model_size_in_b": 7},
    "llama2_chat-13B": {"model_type": "generalist open", "model_size_in_b": 13},
    "llama2_chat-70B": {"model_type": "generalist open", "model_size_in_b": 70},
    "llama3_instruct-8B": {"model_type": "generalist open", "model_size_in_b": 8},
    "llama3_instruct-70B": {"model_type": "generalist open", "model_size_in_b": 70},
    "med42-8B": {"model_type": "biomedical open", "model_size_in_b": 8},
    "med42-70B": {"model_type": "biomedical open", "model_size_in_b": 70},
    "mistral_instruct7B": {"model_type": "generalist open", "model_size_in_b": 7},
    "olmo2_instruct-7B": {"model_type": "generalist open", "model_size_in_b": 7},
    "olmo2_instruct-13B": {"model_type": "generalist open", "model_size_in_b": 13},
    "openbiollm-8B": {"model_type": "biomedical open", "model_size_in_b": 8},
    "openbiollm-70B": {"model_type": "biomedical open", "model_size_in_b": 70}
}

## Spin Detection Task

In [113]:
# detection_overall_metrics.json was manually created
detection_stats_df = pd.read_json("./eval_outputs/detection_overall_metrics.json", orient="index")

detection_stats_df["model_name"] = detection_stats_df.index
detection_stats_df["model_type"] = detection_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
detection_stats_df["model_size_in_b"] = detection_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
detection_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(detection_stats_df)}")

detection_stats_df.sort_index(inplace=True) # alphabetical order
detection_stats_df

Number of models: 22


,accuracy,precision,recall,f1,model_name,model_type,model_size_in_b
0,0.783333,0.707317,0.966667,0.816901,gpt4o,generalist closed,NaN
1,0.850000,0.783784,0.966667,0.865672,gpt4o-mini,generalist closed,NaN
2,0.516667,1.000000,0.033333,0.064516,gpt35,generalist closed,175.0
3,0.733333,0.652174,1.000000,0.789474,gemini_1.5_flash,generalist closed,NaN
4,0.833333,0.794118,0.900000,0.843750,gemini_1.5_flash-8B,generalist closed,8.0
5,0.966667,1.000000,0.933333,0.965517,claude_3.5-sonnet,generalist closed,NaN
6,0.566667,0.535714,1.000000,0.697674,claude_3.5-haiku,generalist closed,NaN
7,0.516667,1.000000,0.033333,0.064516,biomistral7B,biomedical open,7.0
8,0.566667,0.537037,0.966667,0.690476,llama2_chat-13B,generalist open,13.0
9,0.633333,0.580000,0.966667,0.725000,llama2_chat-70B,generalist open,70.0


### Average of accuracy by model type

In [114]:
# Group by model type and calculate mean accuracy and standard deviation
accuracy_by_model_type = detection_stats_df.groupby('model_type')['accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
accuracy_by_model_type.columns = ['model_type', 'mean_accuracy', 'std_deviation']

print(accuracy_by_model_type)

In [115]:
# Group by model type and calculate mean precision and standard deviation
precision_by_model_type = detection_stats_df.groupby('model_type')['precision'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
precision_by_model_type.columns = ['model_type', 'mean_precision', 'std_deviation']

print(precision_by_model_type)

          model_type  mean_precision  std_deviation
0    biomedical open        0.812759       0.219535
1  generalist closed        0.781872       0.172379
2    generalist open        0.702130       0.364676


In [116]:
# Group by model type and calculate mean recall and standard deviation
recall_by_model_type = detection_stats_df.groupby('model_type')['recall'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
recall_by_model_type.columns = ['model_type', 'mean_recall', 'std_deviation']

print(recall_by_model_type)

          model_type  mean_recall  std_deviation
0    biomedical open     0.576190       0.383799
1  generalist closed     0.828571       0.352467
2    generalist open     0.587500       0.406666


In [117]:
# Group by model type and calculate mean f1 and standard deviation
f1_by_model_type = detection_stats_df.groupby('model_type')['f1'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
f1_by_model_type.columns = ['model_type', 'mean_f1', 'std_deviation']

print(f1_by_model_type)

          model_type   mean_f1  std_deviation
0    biomedical open  0.547278       0.275737
1  generalist closed  0.720501       0.300329
2    generalist open  0.539761       0.322221


#### Plots

In [118]:
# Create a dictionary for custom labels
custom_labels = {
    "alpacare-7B": "AlpaCare 7B",
    "biomedgpt7B": "BioMedGPT 7B",
    "biomistral7B": "BioMistral 7B",
    "claude_3.5-haiku": "Claude3.5 Haiku",
    "claude_3.5-sonnet": "Claude3.5 Sonnet", 
    "gemini_1.5_flash": "Gemini1.5 Flash",
    "gemini_1.5_flash-8B": "Gemini1.5 Flash 8B",
    "gpt4o": "GPT4o",
    "gpt4o-mini": "GPT4o Mini",
    "gpt35": "GPT3.5",
    "llama2_chat-7B": "Llama2 Chat 7B",
    "llama2_chat-13B": "Llama2 Chat 13B",
    "llama2_chat-70B": "Llama2 Chat 70B",
    "llama3_instruct-8B": "Llama3 Instruct 8B",
    "llama3_instruct-70B": "Llama3 Instruct 70B",
    "med42-8B": "Med42 8B",
    "med42-70B": "Med42 70B",
    "mistral_instruct7B": "Mistral Instruct 7B",
    "olmo2_instruct-7B": "Olmo2 Instruct 7B",
    "olmo2_instruct-13B": "Olmo2 Instruct 13B",
    "openbiollm-8B": "OpenBioLLM 8B",
    "openbiollm-70B": "OpenBioLLM 70B"
}

detection_stats_df['model_name_custom'] = detection_stats_df['model_name'].map(custom_labels)

color_mapping = {
    'biomedical open': '#0868ac', 
    'generalist closed': '#7bccc4',
    'generalist open': '#bae4bc',
}

# Create the bar chart
chart = alt.Chart(detection_stats_df).mark_bar().encode(
    y=alt.Y('model_name_custom:N', sort='-x', title='Model Name'),
    x=alt.X('accuracy:Q', title='Accuracy'),
    color=alt.Color('model_type:N', title='Model Type',
                    scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
                    legend=alt.Legend(
                    orient='none',
                    legendX=130, legendY=-45,
                    direction='horizontal',
                    titleAnchor='middle'))  # Legend at the bottom
).properties(
    width=800,
)

# Add value labels with increased font size
text = chart.mark_text(
    align='center',
    baseline='middle',
    fontWeight='bold',
    dx=20  # Adjust the position of the text
).encode(
    text=alt.Text('accuracy:Q', format='.2f'),
    color=alt.value('black'),
)

# Add a mean rule
avg_rule = alt.Chart(detection_stats_df).mark_rule(color='red').encode(
    x='mean(accuracy):Q',
    size=alt.value(2)
)

# Add a 50% chance rule
chance_rule = alt.Chart(detection_stats_df).mark_rule(color='gray').encode(
    x='min(accuracy):Q',
    size=alt.value(2),
    strokeDash=alt.value([10, 10])
)

# Increase font size for axis labels, titles, and other components
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Title and facet headers (if any)
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Combine chart and text, and apply the config
c_t = chart + avg_rule + chance_rule + text
c_t = c_t.configure(**chart_config)  # Apply the global configuration

# Save to HTML
c_t.save("./plots/detection_accuracy_by_model.html")

# Display the chart
c_t

alt.LayerChart(...)

In [119]:
# # Create the bar chart
# chart = alt.Chart(detection_stats_df).mark_bar().encode(
#     y=alt.Y('model_name_custom:N', sort='-x', title='Model Name'),
#     x=alt.X('accuracy:Q', title='Accuracy'),
#    color=alt.Color('model_type:N', title='Model Type', legend=alt.Legend(
#         orient='none',
#         legendX=130, legendY=-45,
#         direction='horizontal',
#         titleAnchor='middle'), scale=alt.Scale(range=["#808080", "#A9A9A9", "#D3D3D3", "#BEBEBE"]))  # Legend at the bottom
# ).properties(
#     width=800,
# )

# # Add value labels with increased font size
# text = chart.mark_text(
#     align='center',
#     baseline='middle',
#     fontWeight='bold',
#     dx=18  # Adjust the position of the text
# ).encode(
#     text=alt.Text('accuracy:Q', format='.2f'),
#     color=alt.value('black')  # Set text color to black
# )

# # Add a mean rule
# rule = alt.Chart(detection_stats_df).mark_rule(color='gray').encode(
#     x='mean(accuracy):Q',
#     size=alt.value(2)
# )

# # Increase font size for axis labels, titles, and other components
# chart_config = {
#     "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
#     "header": {"labelFontSize": 20, "titleFontSize": 22},  # Title and facet headers (if any)
#     "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
#     "text": {"fontSize": 20},  # Text mark size
# }

# # Combine chart and text, and apply the config
# c_t = chart + rule + text
# c_t = c_t.configure(**chart_config)  # Apply the global configuration

# # Save to HTML
# c_t.save("./plots/detection_accuracy_by_model_gray.html")

# # Display the chart
# c_t

In [120]:
# Plot average accuracy by model_type and add error bars
bars = alt.Chart(detection_stats_df).mark_bar().encode(
    x=alt.X('model_type:N', title='Model Type', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('mean(accuracy):Q', title='Mean Accuracy'),
    color=alt.Color('model_type:N', title='Model Type', legend=None)
).properties(
    title='Average Accuracy by Model Type',
    width=800  # Set the width to 800 pixels
)

error_bars = alt.Chart(detection_stats_df).mark_errorbar(extent='stdev').encode(
    x=alt.X('model_type:N'),
    y=alt.Y('accuracy:Q')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Adjust the position of the text
).encode(
    text=alt.Text('mean(accuracy):Q', format='.2f')
)

alt.layer(bars, error_bars, text)

alt.LayerChart(...)

### Average of accuracy by model size

In [121]:
# model size in buckets (0-10B, 11-20B, 22-100B, 100B+/NaN)
def model_size_bucket(model_size): 
    if model_size is None or pd.isna(model_size):
        return "Unknown"
    elif model_size >= 100:
        return "100B+"
    elif model_size <= 10:
        return "0-10B"
    elif model_size <= 20:
        return "11-20B"
    else:
        return "21-100B"

In [122]:
# average accuracy by model size
detection_stats_df["model_size_bucket"] = detection_stats_df["model_size_in_b"].map(model_size_bucket)

accuracy_by_model_size = detection_stats_df.groupby('model_size_bucket')['accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
accuracy_by_model_size.columns = ['model_size_bucket', 'mean_accuracy', 'std_deviation']

print(accuracy_by_model_size)

In [123]:
# average precision by model size
precision_by_model_size = detection_stats_df.groupby('model_size_bucket')['precision'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
precision_by_model_size.columns = ['model_size_bucket', 'mean_precision', 'std_deviation']

print(precision_by_model_size)

  model_size_bucket  mean_precision  std_deviation
0             0-10B        0.721719       0.337294
1             100B+        1.000000            NaN
2            11-20B        0.768519       0.327364
3           21-100B        0.836558       0.180942
4           Unknown        0.735798       0.173164


In [124]:
# average recall by model size
recall_by_model_size = detection_stats_df.groupby('model_size_bucket')['recall'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
recall_by_model_size.columns = ['model_size_bucket', 'mean_recall', 'std_deviation']

print(recall_by_model_size)

  model_size_bucket  mean_recall  std_deviation
0             0-10B     0.553333       0.404969
1             100B+     0.033333            NaN
2            11-20B     0.500000       0.659966
3           21-100B     0.775000       0.142400
4           Unknown     0.973333       0.027889


In [125]:
# average f1 score by model size 
f1_by_model_size_bucket = detection_stats_df.groupby('model_size_bucket')['f1'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
f1_by_model_size_bucket.columns = ['model_size_bucket', 'mean_f1', 'std_deviation']

print(f1_by_model_size_bucket)

  model_size_bucket   mean_f1  std_deviation
0             0-10B  0.511597       0.294719
1             100B+  0.064516            NaN
2            11-20B  0.377496       0.442621
3           21-100B  0.780454       0.043987
4           Unknown  0.827048       0.098638


#### Plots

In [126]:
bars = alt.Chart(detection_stats_df).mark_bar().encode(
    x=alt.X('model_name:N', sort='-y', title='Model Name'),
    y=alt.Y('accuracy:Q', title='Accuracy'),
    color=alt.Color('model_size_bucket:N', title='Model Size Bucket')
).properties(
    title='Accuracy by Model Size Bucket',
    width=800,
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Adjust the position of the text
).encode(
    text=alt.Text('accuracy:Q', format='.2f')
)

bars + text

alt.LayerChart(...)

In [127]:
# Plot average accuracy by model_size_bucket and add error bars
bars = alt.Chart(detection_stats_df).mark_bar().encode(
    x=alt.X('model_size_bucket:N', title='Model Size Bucket', axis=alt.Axis(labelAngle=0), sort=['0-10B', '11-20B', '21-100B', '100B+', 'Unknown']),
    y=alt.Y('mean(accuracy):Q', title='Mean Accuracy'),
    color=alt.Color('model_size_bucket:N', title='Model Size Bucket', legend=None)
).properties(
    title='Average Accuracy by Model Size',
    width=800  # Set the width to 800 pixels
)

error_bars = alt.Chart(detection_stats_df).mark_errorbar(extent='stdev').encode(
    x=alt.X('model_size_bucket:N', sort=['0-10B', '11-20B', '21-100B', '100B+', 'Unknown']),
    y=alt.Y('accuracy:Q')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Adjust the position of the text
).encode(
    text=alt.Text('mean(accuracy):Q', format='.2f')
)

alt.layer(bars, error_bars, text)

alt.LayerChart(...)

In [128]:
# scatter plot of model size vs accuracy with model names as labels
scatter_plot = alt.Chart(detection_stats_df).mark_circle().encode(
    x=alt.X('model_size_in_b:Q', title='Model Size (in Billion Parameters)'),
    y=alt.Y('accuracy:Q', title='Accuracy'),
    color=alt.Color('model_type:N', title='Model Type')
).properties(
    title='Model Size vs Accuracy',
    width=800,  # Set the width to 800 pixels
    height=400  # Set the height to 400 pixels
)

text = scatter_plot.mark_text(
    align='left',
    baseline='middle',
    dx=7,  # Adjust the position of the text
    dy=-5,  # Adjust the vertical position of the text
).encode(
    text='model_name:N'
)

scatter_plot + text

alt.LayerChart(...)

## RCT Trial Result Interpretation Task

In [129]:
# interpretation_overall_metrics.json was manually created
interpretation_stats_df = pd.read_json("./eval_outputs/interpretation_overall_metrics.json", orient="index")

interpretation_stats_df["model_name"] = interpretation_stats_df.index
interpretation_stats_df["model_type"] = interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
interpretation_stats_df["model_size_in_b"] = interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(interpretation_stats_df)}")

interpretation_stats_df.sort_index(inplace=True) # alphabetical order
interpretation_stats_df

Number of models: 22


,benefit_answer_mean_diff,rigor_answer_mean_diff,importance_answer_mean_diff,full_text_answer_mean_diff,another_trial_answer_mean_diff,overall_mean_diff_avg,model_name,model_type,model_size_in_b
0,3.133333,0.100000,1.233333,2.866667,3.333333,2.133333,gpt4o,generalist closed,NaN
1,3.566667,1.466667,2.733333,3.933333,3.866667,3.113333,gpt4o-mini,generalist closed,NaN
2,3.900000,1.433333,2.066667,2.600000,3.766667,2.753333,gpt35,generalist closed,175.0
3,2.500000,-0.100000,2.166667,3.000000,3.700000,2.253333,gemini_1.5_flash,generalist closed,NaN
4,3.066667,-0.100000,0.966667,2.733333,3.433333,2.020000,gemini_1.5_flash-8B,generalist closed,8.0
5,2.500000,-0.166667,-0.633333,3.233333,2.866667,1.560000,claude_3.5-sonnet,generalist closed,NaN
6,2.966667,-0.033333,0.466667,1.300000,2.166667,1.373333,claude_3.5-haiku,generalist closed,NaN
7,6.051724,0.266667,0.800000,0.000000,NaN,NaN,alpacare-7B,biomedical open,7.0
8,1.666667,0.350000,1.116667,1.035714,1.666667,1.167143,biomistral7B,biomedical open,7.0
9,3.500000,0.500000,1.066667,0.333333,1.700000,1.420000,llama2_chat-7B,generalist open,7.0


In [130]:
human_expert_stats = {
        "benefit_answer": {"mean_diff": 0.71, "ci_lower": 0.07, "ci_upper": 1.35},
        "rigor_answer": {"mean_diff": -0.59, "ci_lower": -1.13, "ci_upper": -0.05},
        "importance_answer": {"mean_diff": -0.38, "ci_lower": -0.95, "ci_upper": 0.19},
        "full_text_answer": {"mean_diff": 0.77, "ci_lower": 0.08, "ci_upper": 1.47},
        "another_trial_answer": {"mean_diff": 0.64, "ci_lower": -0.03, "ci_upper": 1.31}
    }

human_expert_stats_df = pd.DataFrame(human_expert_stats).T
human_expert_stats_df["metric"] = human_expert_stats_df.index
# remove index
human_expert_stats_df.reset_index(drop=True, inplace=True)
human_expert_stats_df["method"] = "human experts"

human_expert_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,0.71,0.07,1.35,benefit_answer,human experts
1,-0.59,-1.13,-0.05,rigor_answer,human experts
2,-0.38,-0.95,0.19,importance_answer,human experts
3,0.77,0.08,1.47,full_text_answer,human experts
4,0.64,-0.03,1.31,another_trial_answer,human experts


In [131]:
def calculate_confidence_interval(df, df_column_name):
    mean_diff = df[df_column_name].mean()  # Calculate the mean
    std_dev = df[df_column_name].std()  # Calculate the standard deviation
    n = len(df[df_column_name])  # Sample size

    # Calculate the margin of error for 95% CI (z = 1.96)
    z = 1.96
    margin_of_error = z * (std_dev / sqrt(n))

    # Calculate the 95% Confidence Interval
    ci_lower = mean_diff - margin_of_error
    ci_upper = mean_diff + margin_of_error

    return ci_lower, ci_upper

In [132]:
def calculate_model_stats(interpretation_stats_df, method_name = "all LLMs"):
    # calculate the average of all model metrics and calculate 95% CI
    average_model_benefit = interpretation_stats_df["benefit_answer_mean_diff"].mean()
    ci_lower_model_benefit, ci_upper_model_benefit = calculate_confidence_interval(interpretation_stats_df, "benefit_answer_mean_diff")

    average_model_rigor = interpretation_stats_df["rigor_answer_mean_diff"].mean()
    ci_lower_model_rigor, ci_upper_model_rigor = calculate_confidence_interval(interpretation_stats_df, "rigor_answer_mean_diff")

    average_model_importance = interpretation_stats_df["importance_answer_mean_diff"].mean()
    ci_lower_model_importance, ci_upper_model_importance = calculate_confidence_interval(interpretation_stats_df, "importance_answer_mean_diff")

    average_model_full_text = interpretation_stats_df["full_text_answer_mean_diff"].mean()
    ci_lower_model_full_text, ci_upper_model_full_text = calculate_confidence_interval(interpretation_stats_df, "full_text_answer_mean_diff")

    average_model_another_trial = interpretation_stats_df["another_trial_answer_mean_diff"].mean()
    ci_lower_model_another_trial, ci_upper_model_another_trial = calculate_confidence_interval(interpretation_stats_df, "another_trial_answer_mean_diff")

    model_stats = {
        "benefit_answer": {"mean_diff": average_model_benefit, "ci_lower": ci_lower_model_benefit, "ci_upper": ci_upper_model_benefit},
        "rigor_answer": {"mean_diff": average_model_rigor, "ci_lower": ci_lower_model_rigor, "ci_upper": ci_upper_model_rigor},
        "importance_answer": {"mean_diff": average_model_importance, "ci_lower": ci_lower_model_importance, "ci_upper": ci_upper_model_importance},
        "full_text_answer": {"mean_diff": average_model_full_text, "ci_lower": ci_lower_model_full_text, "ci_upper": ci_upper_model_full_text},
        "another_trial_answer": {"mean_diff": average_model_another_trial, "ci_lower": ci_lower_model_another_trial, "ci_upper": ci_upper_model_another_trial}
    }

    model_stats_df = pd.DataFrame(model_stats).T
    model_stats_df["metric"] = model_stats_df.index
    # remove index
    model_stats_df.reset_index(drop=True, inplace=True)
    model_stats_df["method"] = method_name

    return model_stats_df

In [133]:
# calculate the average of all model metrics and calculate 95% CI
model_stats_df = calculate_model_stats(interpretation_stats_df)

model_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,3.381165,2.874680,3.887650,benefit_answer,all LLMs
1,0.277165,0.088599,0.465730,rigor_answer,all LLMs
2,1.264057,0.922720,1.605395,importance_answer,all LLMs
3,2.205034,1.605838,2.804230,full_text_answer,all LLMs
4,2.962500,2.496102,3.428898,another_trial_answer,all LLMs


In [134]:
# get average and 95% CI by model_type from interpretation_stats_df for each model_type
interpretation_stats_df["model_type"] = interpretation_stats_df["model_name"].map(lambda x: model_metadata[x]["model_type"])

# get only generalist closed  models
generalist_closed_model_stats = interpretation_stats_df[interpretation_stats_df["model_type"] == "generalist closed"]

# get only generalist open models
generalist_open_model_stats = interpretation_stats_df[interpretation_stats_df["model_type"] == "generalist open"]

# get only biomedical open models
biomedical_open_model_stats = interpretation_stats_df[interpretation_stats_df["model_type"] == "biomedical open"]


In [135]:
generalist_closed_model_stats_df = calculate_model_stats(generalist_closed_model_stats, method_name = "generalist closed")

generalist_open_model_stats_df = calculate_model_stats(generalist_open_model_stats, method_name = "generalist open")

biomedical_open_model_stats_df = calculate_model_stats(biomedical_open_model_stats, method_name = "biomedical open")

In [136]:
generalist_closed_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,3.090476,2.708265,3.472687,benefit_answer,generalist closed
1,0.371429,-0.177865,0.920723,rigor_answer,generalist closed
2,1.285714,0.433531,2.137898,importance_answer,generalist closed
3,2.809524,2.219158,3.399889,full_text_answer,generalist closed
4,3.304762,2.857216,3.752308,another_trial_answer,generalist closed


In [137]:
generalist_open_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,3.607040,2.784306,4.429774,benefit_answer,generalist open
1,0.291667,0.077190,0.506143,rigor_answer,generalist open
2,1.462500,0.987437,1.937563,importance_answer,generalist open
3,2.308333,1.180117,3.436550,full_text_answer,generalist open
4,2.620833,1.978924,3.262743,another_trial_answer,generalist open


In [138]:
biomedical_open_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,3.413711,2.117145,4.710276,benefit_answer,biomedical open
1,0.166327,0.046496,0.286157,rigor_answer,biomedical open
2,1.015608,0.584498,1.446719,importance_answer,biomedical open
3,1.482489,0.330245,2.634733,full_text_answer,biomedical open
4,3.030000,1.659212,4.400788,another_trial_answer,biomedical open


In [139]:
average_by_model_type = pd.concat([generalist_closed_model_stats_df, generalist_open_model_stats_df, biomedical_open_model_stats_df], ignore_index=True)

average_by_model_type

,mean_diff,ci_lower,ci_upper,metric,method
0,3.090476,2.708265,3.472687,benefit_answer,generalist closed
1,0.371429,-0.177865,0.920723,rigor_answer,generalist closed
2,1.285714,0.433531,2.137898,importance_answer,generalist closed
3,2.809524,2.219158,3.399889,full_text_answer,generalist closed
4,3.304762,2.857216,3.752308,another_trial_answer,generalist closed
5,3.607040,2.784306,4.429774,benefit_answer,generalist open
6,0.291667,0.077190,0.506143,rigor_answer,generalist open
7,1.462500,0.987437,1.937563,importance_answer,generalist open
8,2.308333,1.180117,3.436550,full_text_answer,generalist open
9,2.620833,1.978924,3.262743,another_trial_answer,generalist open


In [140]:
#combine all the dataframes
model_stats_final_df = pd.concat([human_expert_stats_df, model_stats_df, average_by_model_type], ignore_index=True)
#drop "_answer" from the values in metric column
model_stats_final_df['metric'] = model_stats_final_df['metric'].str.replace('_answer', '')

model_stats_final_df

,mean_diff,ci_lower,ci_upper,metric,method
0,0.710000,0.070000,1.350000,benefit,human experts
1,-0.590000,-1.130000,-0.050000,rigor,human experts
2,-0.380000,-0.950000,0.190000,importance,human experts
3,0.770000,0.080000,1.470000,full_text,human experts
4,0.640000,-0.030000,1.310000,another_trial,human experts
5,3.381165,2.874680,3.887650,benefit,all LLMs
6,0.277165,0.088599,0.465730,rigor,all LLMs
7,1.264057,0.922720,1.605395,importance,all LLMs
8,2.205034,1.605838,2.804230,full_text,all LLMs
9,2.962500,2.496102,3.428898,another_trial,all LLMs


### Plots

In [141]:
# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit': 'Treatment Benefit',
    'rigor': 'Study Rigor',
    'importance': 'Study Importance',
    'full_text': 'Interest to Read Full-Text',
    'another_trial': 'Interest to Run Another Trial'
}

# Define the desired order for the facets
facet_order = ['Treatment Benefit', 'Study Rigor', 'Study Importance', 'Interest to Read Full-Text', 'Interest to Run Another Trial']

color_mapping = {
    'human experts': '#0868ac', 
    'all LLMs': '#43a2ca',  
    'generalist closed': '#7bccc4',  
    'generalist open': '#bae4bc',  
    'biomedical open': '#E3F4D4'
}

method_order = ['human experts', 'all LLMs', 'generalist closed', 'generalist open', 'biomedical open']

# Apply the mapping as a calculated field
chart_data = model_stats_final_df.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('method:N', title=None, axis=alt.Axis(labelAngle=-45), sort=method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('method:N', title='Method', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=300,  # Set the width to 300 pixels
    height=300  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("method:N", sort=method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 20))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, error_bars, text, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply the global configuration

# save to html
chart.save("./plots/interpretation_by_measures.html")

chart

alt.FacetChart(...)

In [142]:
# same plot as above but for only top-6 spin detection accuracy models compare in regards to interpretation

claude_sonnet_diff_data = "eval_outputs/claude_3.5-sonnet/claude_3.5-sonnet_interpretation_outputs.json"
gpt4o_mini_diff_data = "eval_outputs/gpt4o-mini/gpt4o-mini_interpretation_outputs.json"
gemini_flash_8b_diff_data = "eval_outputs/gemini_1.5_flash-8B/gemini_1.5_flash-8B_interpretation_outputs.json"
llama3_8b_diff_data = "eval_outputs/llama3_instruct-8B/llama3_instruct-8B_interpretation_outputs.json"
llama3_70b_diff_data = "eval_outputs/llama3_instruct-70B/llama3_instruct-70B_interpretation_outputs.json"
openbiollm_70b_diff_data = "eval_outputs/openbiollm-70B/openbiollm-70B_interpretation_outputs.json"

# read all the data
claude_sonnet_data = pd.read_json(claude_sonnet_diff_data, orient="record")
gpt4o_mini_data = pd.read_json(gpt4o_mini_diff_data, orient="record")
gemini_flash_8b_data = pd.read_json(gemini_flash_8b_diff_data, orient="record")
llama3_8b_data = pd.read_json(llama3_8b_diff_data, orient="record")
llama3_70b_data = pd.read_json(llama3_70b_diff_data, orient="record")
openbiollm_70b_data = pd.read_json(openbiollm_70b_diff_data, orient="record")

def calculate_ci(data):
    mean_diff = np.mean(data)  # Calculate the mean
    std_dev = np.std(data, ddof=1)  # Use ddof=1 for sample standard deviation  # Calculate the standard deviation
    n = len(data)  # Sample size

    # Calculate the margin of error for 95% CI (z = 1.96)
    z = 1.96
    margin_of_error = z * (std_dev / sqrt(n))

    # Calculate the 95% Confidence Interval
    ci_lower = mean_diff - margin_of_error
    ci_upper = mean_diff + margin_of_error

    return ci_lower, ci_upper

def get_metrics(df):
    column_names = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer"]
    
    diff_metrics = []
    
    for col in column_names:
        df_copy = df.copy()
        
        # Remove rows where the column has 'Error' or empty string
        mask = df_copy[col].apply(lambda x: isinstance(x, str) and ("Error" in x or x == ""))
        if mask.any():
            error_pmids = df_copy.loc[mask, 'PMID'].unique().tolist()
            print(f"Column '{col}' has some 'Error' or empty string values. Removing these rows...")
            print(f"PMIDs with errors: {error_pmids}")
            df_copy = df_copy[~df_copy['PMID'].isin(error_pmids)]
            print(f"Remaining rows after cleanup: {len(df_copy)}")
        
        # Ensure the column is converted to numeric, coercing errors
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

        # For each group by PMID, get the column values difference (abstract_type == spin minus abstract_type == non-spin)
        grouped = df_copy.pivot_table(index='PMID', columns='abstract_type', values=col, aggfunc='mean')
        diff = grouped['spin'] - grouped['no_spin']
        diffs = diff.tolist()

        # calculate confidence interval
        ci_lower, ci_upper = calculate_ci(diffs)

        diff_metrics.append({
            "metric": col,
            "mean_diff": diff.mean(),
            "ci_lower": ci_lower,
            "ci_upper": ci_upper
        })

    return diff_metrics


# get metrics for each model
# save to dataframe for each model
model_data = {
    "Claude 3.5 Sonnet": claude_sonnet_data,
    "GPT4o Mini": gpt4o_mini_data,
    "Gemini1.5 Flash 8B": gemini_flash_8b_data,
    "Llama3 8B": llama3_8b_data,
    "Llama3 70B": llama3_70b_data,
    "OpenBioLLM 70B": openbiollm_70b_data
}
df_for_chart = pd.DataFrame()
for model_name, df in model_data.items():
    diff_metrics = get_metrics(df)
    # create a dataframe
    df_model = pd.DataFrame(diff_metrics)
    df_model["method"] = model_name
    df_for_chart = pd.concat([df_for_chart, df_model], ignore_index=True)

df_for_chart = pd.concat([df_for_chart, human_expert_stats_df], ignore_index=True)
#drop "_answer" from the values in metric column
df_for_chart['metric'] = df_for_chart['metric'].str.replace('_answer', '')

df_for_chart

Column 'full_text_answer' has some 'Error' or empty string values. Removing these rows...
PMIDs with errors: [11261827, 12177098, 15947110, 16148021, 16314619, 17179098, 18794551, 20087643, 20530276, 21471562, 22112969, 9093724]
Remaining rows after cleanup: 36


,metric,mean_diff,ci_lower,ci_upper,method
0,benefit,2.500000,1.978884,3.021116,Claude 3.5 Sonnet
1,rigor,-0.166667,-0.302308,-0.031026,Claude 3.5 Sonnet
2,importance,-0.633333,-0.872616,-0.394051,Claude 3.5 Sonnet
3,full_text,3.233333,2.679892,3.786775,Claude 3.5 Sonnet
4,another_trial,2.866667,2.210635,3.522698,Claude 3.5 Sonnet
5,benefit,3.566667,2.989786,4.143547,GPT4o Mini
6,rigor,1.466667,1.118410,1.814923,GPT4o Mini
7,importance,2.733333,2.283300,3.183367,GPT4o Mini
8,full_text,3.933333,3.402286,4.464381,GPT4o Mini
9,another_trial,3.866667,3.281804,4.451529,GPT4o Mini


In [143]:
# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit': 'Treatment Benefit',
    'rigor': 'Study Rigor',
    'importance': 'Study Importance',
    'full_text': 'Interest to Read Full-Text',
    'another_trial': 'Interest to Run Another Trial'
}

# Define the desired order for the facets
facet_order = ['Treatment Benefit', 'Study Rigor', 'Study Importance', 'Interest to Read Full-Text', 'Interest to Run Another Trial']

color_mapping = {
    'human experts': '#0868ac', 
    'Claude 3.5 Sonnet': '#43a2ca',  
    'GPT4o Mini': '#7bccc4',  
    'Gemini1.5 Flash 8B': '#bae4bc',  
    'Llama3 8B': '#E3F4D4',  
    'Llama3 70B': '#fdd49e',  
    'OpenBioLLM 70B': '#fdae61'
}

method_order = ['human experts', 'Claude 3.5 Sonnet', 'GPT4o Mini', 'Gemini1.5 Flash 8B', 'Llama3 8B', 'Llama3 70B', 'OpenBioLLM 70B']

# Apply the mapping as a calculated field
chart_data = df_for_chart.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('method:N', title=None, axis=alt.Axis(labelAngle=-45), sort=method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('method:N', title='Method', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=300,  # Set the width to 300 pixels
    height=300  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("method:N", sort=method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 22))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, error_bars, text, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply the global configuration

# save to html
chart.save("./plots/top_6_models_interpretation_by_measures.html")

chart

alt.FacetChart(...)

### Mitigation Strategies Results

In [144]:
model_stats_df['method'] = 'baseline'

In [145]:
# gold_labelled_interpretation_overall_metrics.json was manually created
gold_labelled_interpretation_stats_df = pd.read_json("./eval_outputs/gold_labelled_interpretation_overall_metrics.json", orient="index")

gold_labelled_interpretation_stats_df["model_name"] = gold_labelled_interpretation_stats_df.index
gold_labelled_interpretation_stats_df["model_type"] = gold_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
gold_labelled_interpretation_stats_df["model_size_in_b"] = gold_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
gold_labelled_interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(gold_labelled_interpretation_stats_df)}")

gold_labelled_interpretation_stats_df.sort_index(inplace=True) # alphabetical order

In [146]:
# calculate the average of all model metrics and calculate 95% CI
gold_labelled_model_stats_df = calculate_model_stats(gold_labelled_interpretation_stats_df, method_name = "+ ref labels")
gold_labelled_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,1.939576,1.467239,2.411913,benefit_answer,+ ref labels
1,-1.432680,-1.804166,-1.061195,rigor_answer,+ ref labels
2,-0.446717,-0.918034,0.024599,importance_answer,+ ref labels
3,0.287751,-0.257165,0.832668,full_text_answer,+ ref labels
4,1.094828,0.541371,1.648284,another_trial_answer,+ ref labels


In [147]:
# model_output_labelled_interpretation_overall_metrics.json was manually created
model_output_labelled_interpretation_stats_df = pd.read_json("./eval_outputs/model_output_labelled_interpretation_overall_metrics.json", orient="index")

model_output_labelled_interpretation_stats_df["model_name"] = model_output_labelled_interpretation_stats_df.index
model_output_labelled_interpretation_stats_df["model_type"] = model_output_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
model_output_labelled_interpretation_stats_df["model_size_in_b"] = model_output_labelled_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
model_output_labelled_interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(model_output_labelled_interpretation_stats_df)}")

model_output_labelled_interpretation_stats_df.sort_index(inplace=True) # alphabetical order

In [148]:
# calculate the average of all model metrics and calculate 95% CI
model_output_labelled_model_stats_df = calculate_model_stats(model_output_labelled_interpretation_stats_df, method_name = "+ model labels")
model_output_labelled_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,2.588531,2.070864,3.106198,benefit_answer,+ model labels
1,-0.251387,-0.426146,-0.076627,rigor_answer,+ model labels
2,0.771471,0.488157,1.054785,importance_answer,+ model labels
3,1.357926,0.830169,1.885683,full_text_answer,+ model labels
4,2.147018,1.642268,2.651768,another_trial_answer,+ model labels


In [149]:
# combined_detection_interpretation_overall_metrics.json was manually created
combined_interpretation_stats_df = pd.read_json("./eval_outputs/combined_detection_interpretation_overall_metrics.json", orient="index")

combined_interpretation_stats_df["model_name"] = combined_interpretation_stats_df.index
combined_interpretation_stats_df["model_type"] = combined_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_type"])
combined_interpretation_stats_df["model_size_in_b"] = combined_interpretation_stats_df.index.map(lambda x: model_metadata[x]["model_size_in_b"])
# remove index
combined_interpretation_stats_df.reset_index(drop=True, inplace=True)

print(f"Number of models: {len(combined_interpretation_stats_df)}")

combined_interpretation_stats_df.sort_index(inplace=True) # alphabetical order
# combined_interpretation_stats_df

In [150]:
# calculate the average of all model metrics and calculate 95% CI
combined_interpretation_stats_df = calculate_model_stats(combined_interpretation_stats_df, method_name = "detect + interpret")
combined_interpretation_stats_df

,mean_diff,ci_lower,ci_upper,metric,method
0,1.140562,0.690811,1.590312,benefit_answer,detect + interpret
1,-0.643669,-1.143978,-0.143359,rigor_answer,detect + interpret
2,0.673458,0.191343,1.155573,importance_answer,detect + interpret
3,0.261905,-0.386231,0.910040,full_text_answer,detect + interpret
4,1.118410,0.642599,1.594220,another_trial_answer,detect + interpret


In [151]:
all_results = pd.concat([human_expert_stats_df, model_stats_df, gold_labelled_model_stats_df, model_output_labelled_model_stats_df, combined_interpretation_stats_df], ignore_index=True)

all_results

,mean_diff,ci_lower,ci_upper,metric,method
0,0.710000,0.070000,1.350000,benefit_answer,human experts
1,-0.590000,-1.130000,-0.050000,rigor_answer,human experts
2,-0.380000,-0.950000,0.190000,importance_answer,human experts
3,0.770000,0.080000,1.470000,full_text_answer,human experts
4,0.640000,-0.030000,1.310000,another_trial_answer,human experts
5,3.381165,2.874680,3.887650,benefit_answer,baseline
6,0.277165,0.088599,0.465730,rigor_answer,baseline
7,1.264057,0.922720,1.605395,importance_answer,baseline
8,2.205034,1.605838,2.804230,full_text_answer,baseline
9,2.962500,2.496102,3.428898,another_trial_answer,baseline


In [152]:
# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit_answer': 'Treatment Benefit',
    'rigor_answer': 'Study Rigor',
    'importance_answer': 'Study Importance',
    'full_text_answer': 'Interest to Read Full-Text',
    'another_trial_answer': 'Interest to Run Another Trial'
}

# Define the desired order for the facets
facet_order = ['Treatment Benefit', 'Study Rigor', 'Study Importance', 'Interest to Read Full-Text', 'Interest to Run Another Trial']

color_mapping = {
    'human experts': '#0868ac',  
    'baseline': '#43a2ca',  
    '+ ref labels': '#7bccc4',  
    '+ model labels': '#bae4bc', 
    'detect + interpret': '#E3F4D4'  
}

method_order = ['human experts', 'baseline', '+ ref labels', '+ model labels', 'detect + interpret']

# Apply the mapping as a calculated field
chart_data = all_results.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('method:N', title=None, axis=alt.Axis(labelAngle=-45), sort = method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('method:N', title='Method', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=300,  # Set the width to 300 pixels
    height=300  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("method:N", sort = method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 22))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, error_bars, text, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply the global configuration

# save to html
chart.save("./plots/interpretation_by_measures_all_methods.html")

chart

alt.FacetChart(...)

In [182]:
interpretation_stats_df["method_category"] = "baseline"
gold_labelled_interpretation_stats_df["method_category"] = "gold_labelled"
model_output_labelled_interpretation_stats_df["method_category"] = "model_output_labelled"

all_interpretation_stats_df = pd.concat([interpretation_stats_df, gold_labelled_interpretation_stats_df, model_output_labelled_interpretation_stats_df], ignore_index=True)

all_interpretation_stats_df

,benefit_answer_mean_diff,rigor_answer_mean_diff,importance_answer_mean_diff,full_text_answer_mean_diff,another_trial_answer_mean_diff,overall_mean_diff_avg,model_name,model_type,model_size_in_b,method_category,overall_avg
0,3.133333,0.100000,1.233333,2.866667,3.333333,2.133333,gpt4o,generalist closed,NaN,baseline,NaN
1,3.566667,1.466667,2.733333,3.933333,3.866667,3.113333,gpt4o-mini,generalist closed,NaN,baseline,NaN
2,3.900000,1.433333,2.066667,2.600000,3.766667,2.753333,gpt35,generalist closed,175.0,baseline,NaN
3,2.500000,-0.100000,2.166667,3.000000,3.700000,2.253333,gemini_1.5_flash,generalist closed,NaN,baseline,NaN
4,3.066667,-0.100000,0.966667,2.733333,3.433333,2.020000,gemini_1.5_flash-8B,generalist closed,8.0,baseline,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61,4.633333,-0.233333,1.366667,1.900000,2.766667,NaN,olmo2_instruct-13B,generalist open,13.0,model_output_labelled,2.086667
62,3.304348,-0.068966,1.518519,2.518519,1.800000,NaN,openbiollm-8B,biomedical open,8.0,model_output_labelled,1.814484
63,3.833333,-0.183333,1.000000,0.615385,4.733333,NaN,openbiollm-70B,biomedical open,70.0,model_output_labelled,1.999744
64,2.866667,-0.133333,1.533333,2.600000,2.333333,NaN,mistral_instruct7B,generalist open,7.0,model_output_labelled,1.840000


In [183]:
# fit the model
model = smf.ols(formula="benefit_answer_mean_diff ~ method_category", 
                            data=all_interpretation_stats_df)
results = model.fit()

print(results.summary())


tukey_oneway = pairwise_tukeyhsd(endog = all_interpretation_stats_df["benefit_answer_mean_diff"], groups = all_interpretation_stats_df["method_category"])

# Display the results
tukey_oneway.summary()

                               OLS Regression Results                               
Dep. Variable:     benefit_answer_mean_diff   R-squared:                       0.203
Model:                                  OLS   Adj. R-squared:                  0.178
Method:                       Least Squares   F-statistic:                     8.036
Date:                      Sun, 16 Mar 2025   Prob (F-statistic):           0.000779
Time:                              21:24:49   Log-Likelihood:                -103.85
No. Observations:                        66   AIC:                             213.7
Df Residuals:                            63   BIC:                             220.3
Df Model:                                 2                                         
Covariance Type:                  nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

group1,group2,meandiff,p-adj,lower,upper,reject
baseline,gold_labelled,-1.4416,0.0005,-2.3062,-0.577,True
baseline,model_output_labelled,-0.7926,0.0789,-1.6572,0.0719,False
gold_labelled,model_output_labelled,0.649,0.1773,-0.2156,1.5135,False


In [212]:
# Create forest plot that shows the difference in scores between baseline and detect + interpret method for treatment benefit

baseline_stats_df = pd.read_json("./eval_outputs/interpretation_overall_metrics.json", orient="index")
baseline_stats_df["model_name"] = baseline_stats_df.index
baseline_stats_df.drop(["overall_mean_diff_avg", "rigor_answer_mean_diff", "importance_answer_mean_diff", "full_text_answer_mean_diff", "another_trial_answer_mean_diff"], axis=1, inplace=True)
# remove index
baseline_stats_df.reset_index(drop=True, inplace=True)
# rename column name benefit_answer_mean_diff to baseline_benefit
baseline_stats_df.rename(columns={"benefit_answer_mean_diff": "baseline_benefit"}, inplace=True)

combined_interpretation_stats_df = pd.read_json("./eval_outputs/combined_detection_interpretation_overall_metrics.json", orient="index")
combined_interpretation_stats_df["model_name"] = combined_interpretation_stats_df.index
combined_interpretation_stats_df.drop(["overall_avg", "rigor_answer_mean_diff", "importance_answer_mean_diff", "full_text_answer_mean_diff", "another_trial_answer_mean_diff"], axis=1, inplace=True)
# remove index
combined_interpretation_stats_df.reset_index(drop=True, inplace=True)
# rename column name benefit_answer_mean_diff to detect_interpret_benefit
combined_interpretation_stats_df.rename(columns={"benefit_answer_mean_diff": "detect_interpret_benefit"}, inplace=True)

# merge the dataframes by model_name
combined_methods_stats_df = pd.merge(baseline_stats_df, combined_interpretation_stats_df, on="model_name")
combined_methods_stats_df["model_name_custom"] = combined_methods_stats_df["model_name"].map(custom_labels)

combined_methods_stats_df


,baseline_benefit,model_name,detect_interpret_benefit,model_name_custom
0,3.133333,gpt4o,0.900000,GPT4o
1,3.566667,gpt4o-mini,1.333333,GPT4o Mini
2,3.900000,gpt35,2.533333,GPT3.5
3,2.500000,gemini_1.5_flash,0.100000,Gemini1.5 Flash
4,3.066667,gemini_1.5_flash-8B,0.900000,Gemini1.5 Flash 8B
5,2.500000,claude_3.5-sonnet,0.866667,Claude3.5 Sonnet
6,2.966667,claude_3.5-haiku,0.733333,Claude3.5 Haiku
7,6.051724,alpacare-7B,-1.285714,AlpaCare 7B
8,1.666667,biomistral7B,1.000000,BioMistral 7B
9,3.500000,llama2_chat-7B,1.666667,Llama2 Chat 7B


In [237]:
model_order = ['Claude3.5 Sonnet', 
    'GPT4o Mini', 'Gemini1.5 Flash 8B', 'Llama3 Instruct 8B', 'Llama3 Instruct 70B', 'OpenBioLLM 70B', 'Med42 70B',
    'GPT4o', 'Gemini1.5 Flash', 'Olmo2 Instruct 7B', 'AlpaCare 7B', 'Llama2 Chat 70B', 'Med42 8B', 'Claude3.5 Haiku', 'Llama2 Chat 13B',
    'GPT3.5', 'BioMistral 7B', 'Olmo2 Instruct 13B', 'OpenBioLLM 8B', 'Mistral Instruct 7B', 'Llama2 Chat 7B', 'BioMedGPT 7B']

bar = alt.Chart(combined_methods_stats_df).mark_bar(cornerRadius=10, height=10).encode(
    x=alt.X('detect_interpret_benefit:Q').scale(domain=[-2, 6.5]).title('Mean Difference for Treatment Benefit'),
    x2='baseline_benefit:Q',
    y=alt.Y('model_name_custom:N', title="LLM Name", sort=model_order)  # Specify custom order
)

text_min = alt.Chart(combined_methods_stats_df).mark_text(align='right', dx=-5, fontSize=14, fontWeight='bold').encode(
    x='detect_interpret_benefit:Q',
    y=alt.Y('model_name_custom:N', sort=model_order),
    text=alt.Text('detect_interpret_benefit:Q', format='.2f'),
    color=alt.value('#FF8100')  # Set text color to orange
)

text_max = alt.Chart(combined_methods_stats_df).mark_text(align='left', dx=5, fontSize=14, fontWeight='bold').encode(
    x='baseline_benefit:Q',
    y=alt.Y('model_name_custom:N', sort=model_order),
    text=alt.Text('baseline_benefit:Q', format='.2f')
)

chart = (bar + text_min + text_max).properties(
    width=800,
    config={
        'axis': {
            'labelFontSize': 16,
            'titleFontSize': 18
        }
    })

# # Save to HTML
chart.save("./plots/baseline_detect_interpret_treatment_benefit_diff.html")

chart

alt.LayerChart(...)

## Relationship between spin / spin detection and spin interpretation

Linear Regression with statsmodels Python package

In [155]:
# get all model names
model_names = model_metadata.keys()
# remove alpacare-13B
model_names = [x for x in model_names if x != "alpacare-13B"]

len(model_names)

22

In [156]:
measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer"]
gpt_models = ["gpt4o", "gpt4o-mini", "gpt35"]
huggingface_models = ["alpacare-7B", "biomedgpt7B", "biomistral7B", 
                      "llama2_chat-7B", "llama2_chat-13B", "llama2_chat-70B",
                      "llama3_instruct-8B", "llama3_instruct-70B",
                      "med42-8B", "med42-70B", "mistral_instruct7B", 
                      "olmo2_instruct-7B", "olmo2_instruct-13B",
                      "openbiollm-8B", "openbiollm-70B"]
no_probability_models = ["claude_3.5-haiku", "claude_3.5-sonnet", "gemini_1.5_flash", "gemini_1.5_flash-8B"]

def get_is_detection_correct(row):
    if row['abstract_type'] == "spin":
        return row['model_answer'] == "yes"
    else:
        return row['model_answer'] == "no"
    
def get_is_abstract_type_spin(row):
    return row['abstract_type'] == "spin"
    
def detection_probability_gpt(row):
    # find the first instance of "yes" or "no"
    token_probabilties = row['model_log_probabilities']
    for token_prob in token_probabilties:
        if token_prob['token'].lower() == "yes":
            return np.exp(token_prob['logprob'])
        elif token_prob['token'].lower() == "no":
            return np.exp(token_prob['logprob'])
    return None # this should not happen but just in case

def detection_probability_huggingface(row):
    # find the first instance of "yes" or "no"
    token_probabilties = row['model_log_probabilities']
    for token_prob in token_probabilties:
        if token_prob['token_string'].lower() == "yes":
            return token_prob['probability']
        elif token_prob['token_string'].lower() == "no":
            return token_prob['probability']
    return None # this should not happen but just in case


def prepare_data_for_regression(model_names):
    for model_name in tqdm(model_names):
        # print(f"Processing {model_name}...")
        final_data = []
        detection_output_file_path = f"./eval_outputs/{model_name}/{model_name}_detection_outputs.json"
        interpretation_output_file_path = f"./eval_outputs/{model_name}/{model_name}_interpretation_outputs.json"
        model_detection_data = pd.read_json(detection_output_file_path, orient="records")
        model_interpretation_data = pd.read_json(interpretation_output_file_path, orient="records")

        # merge model_detection_data and model_interpretation_data by PMID and abstract_type
        model_data = pd.merge(model_detection_data, model_interpretation_data, on=['PMID', 'abstract_type'])

        # loop through each row in model_data
        for _, row in model_data.iterrows():
            detection_model_prediction = 1 if row['model_answer'] == "yes" else 0
            is_detection_correct = 1 if get_is_detection_correct(row) else 0
            is_spin_in_abstract = 1 if get_is_abstract_type_spin(row) else 0

            if model_name in gpt_models:
                detection_probability = detection_probability_gpt(row)
            elif model_name in huggingface_models:
                detection_probability = detection_probability_huggingface(row)
            else:
                detection_probability = None
            
            for measure in measures:
                final_data.append({
                    "pmid": row['PMID'],
                    "measure": measure,
                    "is_spin_in_abstract": is_spin_in_abstract,
                    "is_detection_correct": is_detection_correct,
                    "detection_model_prediction": detection_model_prediction,
                    "detection_probability": detection_probability,
                    "interpretation_answer": float(row[measure]) if row[measure] != "" else None
                })
            # calculate the average of the differences
            answers = []
            for measure in measures:
                if row[measure] != "":
                    answers.append(float(row[measure]))
            if len(answers) > 0:
                avg_answer= round(np.mean(answers), 6)
            else:
                avg_answer = None
            # add the average difference to the data
            final_data.append({
                "pmid": row['PMID'],
                "measure": "overall",
                "is_spin_in_abstract": is_spin_in_abstract,
                "is_detection_correct": is_detection_correct,
                "detection_model_prediction": detection_model_prediction,
                "detection_probability": detection_probability,
                "interpretation_answer": avg_answer
            })

        # save the final data to a json file
        json_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
        save_dataset_to_csv(final_data, json_file_path)

In [157]:
prepare_data_for_regression(model_names=model_names)

100%|██████████| 22/22 [00:02<00:00, 10.90it/s]


#### Simplest Regression

Is spin in abstract and the measures answers

In [158]:
for model_name in model_names:
    output_string = ""
    csv_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
    data = pd.read_csv(csv_file_path)

    measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer", "overall"]
    for measure in measures:
        # get the data for the current measure
        measure_data = data[data['measure'] == measure]
        nan_rows_number = measure_data['interpretation_answer'].isnull().sum()
        # remove rows with NaN values in interpretation_answer
        measure_data = measure_data.dropna(subset=['interpretation_answer'])

        # check if there are less than 2 rows
        if len(measure_data) < 2:
            continue
        
        # fit the model
        model = smf.ols(formula="interpretation_answer ~ is_spin_in_abstract", 
                                    data=measure_data)
        results = model.fit()

        output_string += f"Model: {model_name} - {measure}\n"
        # print number of rows with NaN value(s)
        output_string += f"Number of rows with NaN value(s) in {model_name}: {nan_rows_number}\n"
        output_string += results.summary().as_text()
        output_string += "\n"

    # save the model summary
    with open(f"./eval_outputs/{model_name}/{model_name}_simple_regression_summary.txt", "w") as f:
        f.write(output_string)

##### Forest Plot for "Benefit" Linear Regression Results

In [159]:
import altair as alt
import pandas as pd

# Load the JSON data into a DataFrame
# this json file was manually created
regression_results_df = pd.read_json("./eval_outputs/simple_linear_regression_benefit_data.json", orient="index")

# Ensure index is reset and available as a column
regression_results_df.reset_index(inplace=True)
regression_results_df = regression_results_df.rename(columns={'index': 'model_name'})

regression_results_df["model_name_custom"] = regression_results_df["model_name"].map(custom_labels)

# Create the Altair chart
points = alt.Chart(regression_results_df).mark_point(
    filled=True,
    color='red',
    size=50  # Increase point size
).encode(
    x=alt.X('coef:Q').title('Coefficient'),
    y=alt.Y('model_name_custom:N').title('LLM Name').sort(
        field='coef',  # Sort by coefficient values
        order='descending'
    )
).properties(
    width=600,
    height=300
)

# Add error bars
error_bars = points.mark_rule(
    strokeWidth=2  # Increase width of error bars
).encode(
    x='ci_lower:Q',
    x2='ci_upper:Q',
    size=alt.value(2)  # Set the width of error bars
)

# Add vertical line at x = 0.71
vertical_line = alt.Chart(pd.DataFrame({'x': [0.71]})).mark_rule(
    color='blue',
    strokeDash=[4, 4],  # Make it dashed
    strokeWidth=2
).encode(
    x='x:Q',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Add label for vertical line
label = alt.Chart(pd.DataFrame({'x': [0.71], 'y': [regression_results_df['model_name_custom'].iloc[0]]})).mark_text(
    text='Human Experts',
    align='center',
    dx=5,  # Adjust text position
    dy=-10,
    fontSize=14,
    fontWeight='bold',
).encode(
    x='x:Q',
    y=alt.value(0),  # Adjust position if necessary
    color=alt.value('#0868ac')  # Specify the color directly
)

# Define custom x-axis labels
custom_labels_df = pd.DataFrame({
    'x': [regression_results_df['coef'].min(), regression_results_df['coef'].max()],
    'text': ['Less susceptible to spin', 'More susceptible to spin']
})

# Define custom x-axis labels
left_arrow_df = pd.DataFrame({
    'x': [0.2],
    'text': ['←']
})

# Define custom x-axis labels
right_arrow_df = pd.DataFrame({
    'x': [7.7],
    'text': ['→']
})

# Create a text layer for custom x-axis labels
custom_x_labels = alt.Chart(custom_labels_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=24, # adjust horizontal positioning
    fontSize=14,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Create a text layer for custom x-axis labels
custom_x_left_arrow = alt.Chart(left_arrow_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=10, # adjust horizontal positioning
    fontSize=24,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Create a text layer for custom x-axis labels
custom_x_right_arrow = alt.Chart(right_arrow_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=0, # adjust horizontal positioning
    fontSize=24,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Combine all layers, including the new x-axis labels
chart = alt.layer(error_bars, points, vertical_line, label, custom_x_labels, custom_x_left_arrow, custom_x_right_arrow).configure_axis(
    labelFontSize=16,
    titleFontSize=18
).configure_title(
    fontSize=20
)

# # Save to HTML
chart.save("./plots/simple_regression_benefit_data.html")

chart


alt.LayerChart(...)

In [160]:
# FOR KAREN

import altair as alt
import pandas as pd

# TODO: CHANGE FILE NAME HERE
# Load the JSON data into a DataFrame
regression_results_df = pd.read_json("./eval_outputs/simple_linear_regression_benefit_data.json", orient="index")

# Ensure index is reset and available as a column
regression_results_df.reset_index(inplace=True)
regression_results_df = regression_results_df.rename(columns={'index': 'model_name'})

regression_results_df["model_name_custom"] = regression_results_df["model_name"].map(custom_labels)

# Create the Altair chart
points = alt.Chart(regression_results_df).mark_point(
    filled=True,
    color='red',
    size=50  # Increase point size
).encode(
    x=alt.X('coef:Q').title('Coefficient'),
    y=alt.Y('model_name_custom:N').title('LLM Name').sort(
        field='coef',  # Sort by coefficient values
        order='descending'
    )
).properties(
    width=600,
    height=300
)

# Add error bars
error_bars = points.mark_rule(
    strokeWidth=2  # Increase width of error bars
).encode(
    x='ci_lower:Q',
    x2='ci_upper:Q',
    size=alt.value(2)  # Set the width of error bars
)

# Add vertical line at x = 0.71
# TODO: change the x value to what the human expert values are
# 1	-0.590000	rigor_answer	human experts
# 2	-0.380000	importance_answer	human experts
# 3	0.770000	full_text_answer	human experts
# 4	0.640000	another_trial_answer	human experts
vertical_line = alt.Chart(pd.DataFrame({'x': [0.71]})).mark_rule(
    color='blue',
    strokeDash=[4, 4],  # Make it dashed
    strokeWidth=2
).encode(
    x='x:Q',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Add label for vertical line
label = alt.Chart(pd.DataFrame({'x': [0.71], 'y': [regression_results_df['model_name_custom'].iloc[0]]})).mark_text(
    text='Human Experts',
    align='center',
    dx=5,  # Adjust text position
    dy=-10,
    fontSize=14,
    fontWeight='bold',
).encode(
    x='x:Q',
    y=alt.value(0),  # Adjust position if necessary
    color=alt.value('#0868ac')  # Specify the color directly
)

# Define custom x-axis labels
custom_labels_df = pd.DataFrame({
    'x': [regression_results_df['coef'].min(), regression_results_df['coef'].max()],
    'text': ['Less susceptible to spin', 'More susceptible to spin']
})

# TODO: hard coded x-axis for the arrows
# MIGHT NEED TO ADJUST AS NEEDED
# Define custom x-axis labels
left_arrow_df = pd.DataFrame({
    'x': [0.2],
    'text': ['←']
})

# TODO: hard coded x-axis for the arrows
# MIGHT NEED TO ADJUST AS NEEDED
# Define custom x-axis labels
right_arrow_df = pd.DataFrame({
    'x': [7.7],
    'text': ['→']
})

# Create a text layer for custom x-axis labels
custom_x_labels = alt.Chart(custom_labels_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=24, # adjust horizontal positioning
    fontSize=14,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Create a text layer for custom x-axis labels
custom_x_left_arrow = alt.Chart(left_arrow_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=10, # adjust horizontal positioning
    fontSize=24,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Create a text layer for custom x-axis labels
custom_x_right_arrow = alt.Chart(right_arrow_df).mark_text(
    align='center',
    baseline='bottom',
    dy=195,  # Adjust vertical positioning
    dx=0, # adjust horizontal positioning
    fontSize=24,
    fontWeight='bold',
).encode(
    x='x:Q',
    text='text:N',
    color=alt.value('#0868ac')  # Specify the color directly
)

# Combine all layers, including the new x-axis labels
chart = alt.layer(error_bars, points, vertical_line, label, custom_x_labels, custom_x_left_arrow, custom_x_right_arrow).configure_axis(
    labelFontSize=16,
    titleFontSize=18
).configure_title(
    fontSize=20
)

# Save to HTML
# TODO: 
# CHANGE FILE NAME HERE
chart.save("./plots/simple_regression_benefit_data.html")

chart


alt.LayerChart(...)

#### Binary Spin Detection Results Version

In [161]:
# for model_name in model_names:
#     output_string = ""
#     csv_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
#     data = pd.read_csv(csv_file_path)

#     measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer", "overall"]
#     for measure in measures:
#         # get the data for the current measure
#         measure_data = data[data['measure'] == measure]
#         nan_rows_number = measure_data['interpretation_answer'].isnull().sum()
#         # remove rows with NaN values in interpretation_answer
#         measure_data = measure_data.dropna(subset=['interpretation_answer'])

#         # check if there are less than 2 rows
#         if len(measure_data) < 2:
#             continue
        
#         # fit the model
#         model = smf.ols(formula="interpretation_answer ~ is_spin_in_abstract * is_detection_correct", 
#                                     data=measure_data)
#         results = model.fit()

#         output_string += f"Model: {model_name} - {measure}\n"
#         # print number of rows with NaN value(s)
#         output_string += f"Number of rows with NaN value(s) in {model_name}: {nan_rows_number}\n"
#         output_string += results.summary().as_text()
#         output_string += "\n"

#     # save the model summary
#     with open(f"./eval_outputs/{model_name}/{model_name}_regression_binary_summary.txt", "w") as f:
#         f.write(output_string)

In [162]:
# # what the model predicts rather than whether it was correct or not
# for model_name in model_names:
#     output_string = ""
#     csv_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
#     data = pd.read_csv(csv_file_path)

#     measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer", "overall"]
#     for measure in measures:
#         # get the data for the current measure
#         measure_data = data[data['measure'] == measure]
#         nan_rows_number = measure_data['interpretation_answer'].isnull().sum()
#         # remove rows with NaN values in interpretation_answer
#         measure_data = measure_data.dropna(subset=['interpretation_answer'])

#         # check if there are less than 2 rows
#         if len(measure_data) < 2:
#             continue
        
#         # fit the model
#         model = smf.ols(formula="interpretation_answer ~ is_spin_in_abstract * detection_model_prediction", 
#                                     data=measure_data)
#         results = model.fit()

#         output_string += f"Model: {model_name} - {measure}\n"
#         # print number of rows with NaN value(s)
#         output_string += f"Number of rows with NaN value(s) in {model_name}: {nan_rows_number}\n"
#         output_string += results.summary().as_text()
#         output_string += "\n"

#     # save the model summary
#     with open(f"./eval_outputs/{model_name}/{model_name}_regression_binary_direct_model_prediction_summary.txt", "w") as f:
#         f.write(output_string)

#### Probability Spin Detection Results Version

In [163]:
# model_names = gpt_models + huggingface_models # remove no token probability models

# for model_name in model_names:
#     output_string = ""
#     csv_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
#     data = pd.read_csv(csv_file_path)

#     measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer", "overall"]
#     for measure in measures:
#         # get the data for the current measure
#         measure_data = data[data['measure'] == measure]
#         nan_rows_number = measure_data['interpretation_answer'].isnull().sum()
#         # remove rows with NaN values in interpretation_answer
#         measure_data = measure_data.dropna(subset=['interpretation_answer', 'detection_probability'])
        
#         # if is_detection_no_spin_correct == 1, then detection_probability. Otherwise, 1 - detection_probability
#         measure_data['regression_detection_variable'] = measure_data.apply(lambda x: x['detection_probability'] if x['is_detection_correct'] == 1 else 1 - x['detection_probability'], axis=1)
#         # check if there are less than 2 rows
#         if len(measure_data) < 2:
#             continue

#         # fit the model
#         model = smf.ols(formula="interpretation_answer ~ is_spin_in_abstract * regression_detection_variable",
#                                     data=measure_data)
#         results = model.fit()

#         output_string += f"Model: {model_name} - {measure}\n"
#         # print number of rows with NaN value(s)
#         output_string += f"Number of rows with NaN value(s) in {model_name}: {nan_rows_number}\n"
#         output_string += results.summary().as_text()
#         output_string += "\n"

#     # save the model summary
#     with open(f"./eval_outputs/{model_name}/{model_name}_regression_probability_summary.txt", "w") as f:
#         f.write(output_string)

In [164]:
# # what the model predicts rather than whether it was correct or not

# model_names = gpt_models + huggingface_models # remove no token probability models

# for model_name in model_names:
#     output_string = ""
#     csv_file_path = f"./eval_outputs/{model_name}/{model_name}_combined_data.csv"
#     data = pd.read_csv(csv_file_path)

#     measures = ["benefit_answer", "rigor_answer", "importance_answer", "full_text_answer", "another_trial_answer", "overall"]
#     for measure in measures:
#         # get the data for the current measure
#         measure_data = data[data['measure'] == measure]
#         nan_rows_number = measure_data['interpretation_answer'].isnull().sum()
#         # remove rows with NaN values in interpretation_answer
#         measure_data = measure_data.dropna(subset=['interpretation_answer', 'detection_probability'])
        
#         # if is_detection_no_spin_correct == 1, then detection_probability. Otherwise, 1 - detection_probability
#         measure_data['regression_detection_variable'] = measure_data.apply(lambda x: x['detection_probability'] if x['detection_model_prediction'] == 1 else 1 - x['detection_probability'], axis=1)
#         # check if there are less than 2 rows
#         if len(measure_data) < 2:
#             continue

#         # fit the model
#         model = smf.ols(formula="interpretation_answer ~ is_spin_in_abstract * regression_detection_variable",
#                                     data=measure_data)
#         results = model.fit()

#         output_string += f"Model: {model_name} - {measure}\n"
#         # print number of rows with NaN value(s)
#         output_string += f"Number of rows with NaN value(s) in {model_name}: {nan_rows_number}\n"
#         output_string += results.summary().as_text()
#         output_string += "\n"

#     # save the model summary
#     with open(f"./eval_outputs/{model_name}/{model_name}_regression_probability_direct_model_prediction_summary.txt", "w") as f:
#         f.write(output_string)

## PLS LLM Interpretations

### Average Tokens

In [165]:
# get all PLS outputs from all LLMs
enc = tiktoken.get_encoding("o200k_base") # for gpt-4o and gpt-4o mini

model_token_stats = {}
number_of_tokens_total = [] # for all models
for model_name in model_names:
    csv_file_path = f"./pls_outputs/{model_name}/{model_name}_outputs.csv"
    data = pd.read_csv(csv_file_path)
    # calculate the number of tokens for each row in plain_language_summary
    plain_language_summaries = data['plain_language_summary'].tolist()

    number_of_tokens = []
    for summary in plain_language_summaries:
        token_integers = enc.encode(summary)
        number_of_tokens.append(len(token_integers))
        number_of_tokens_total.append(len(token_integers))

    # average number of tokens
    average_number_of_tokens = np.mean(number_of_tokens)
    # SD of tokens
    sd_number_of_tokens = np.std(number_of_tokens)
    model_token_stats[model_name] = {"average_number_of_tokens": average_number_of_tokens, "sd_number_of_tokens": sd_number_of_tokens}

model_token_stats_df = pd.DataFrame(model_token_stats).T
model_token_stats_df["model_name"] = model_token_stats_df.index

model_token_stats_df

,average_number_of_tokens,sd_number_of_tokens,model_name
alpacare-7B,120.416667,56.264936,alpacare-7B
biomedgpt7B,195.000000,59.498459,biomedgpt7B
biomistral7B,140.766667,73.807941,biomistral7B
claude_3.5-haiku,225.033333,15.084171,claude_3.5-haiku
claude_3.5-sonnet,230.516667,19.185491,claude_3.5-sonnet
gemini_1.5_flash,214.216667,30.953778,gemini_1.5_flash
gemini_1.5_flash-8B,207.700000,37.768285,gemini_1.5_flash-8B
gpt4o,207.850000,44.255254,gpt4o
gpt4o-mini,253.783333,38.084595,gpt4o-mini
gpt35,94.483333,18.754992,gpt35


In [166]:
# get the average across all
average_number_of_tokens = np.mean(number_of_tokens_total)
sd_number_of_tokens = np.std(number_of_tokens_total)

average_number_of_tokens, sd_number_of_tokens

(208.10378787878787, 67.01472464306116)

In [167]:
# average across all models
average_number_of_tokens = model_token_stats_df["average_number_of_tokens"].mean()
sd_number_of_tokens = model_token_stats_df["sd_number_of_tokens"].mean()

average_number_of_tokens, sd_number_of_tokens

(208.10378787878787, 37.62843632260712)

### LLM Evaluation Results

In [168]:
# the following files were manually created
claude_evaluator_results = pd.read_json("./pls_outputs/_interpretation_eval_results/claude_3.5-sonnet/claude_3.5-sonnet_interpretation_overall_metrics.json", orient="index")
gpt4o_mini_evaluator_results = pd.read_json("./pls_outputs/_interpretation_eval_results/gpt4o-mini/gpt4o-mini_interpretation_overall_metrics.json", orient="index")

In [169]:
def calculate_confidence_interval(df, df_column_name):
    mean_diff = df[df_column_name].mean()  # Calculate the mean
    std_dev = df[df_column_name].std()  # Calculate the standard deviation
    n = len(df[df_column_name])  # Sample size

    # Calculate the margin of error for 95% CI (z = 1.96)
    z = 1.96
    margin_of_error = z * (std_dev / sqrt(n))

    # Calculate the 95% Confidence Interval
    ci_lower = mean_diff - margin_of_error
    ci_upper = mean_diff + margin_of_error

    return ci_lower, ci_upper

In [170]:
# calculate the average of all model metrics and calculate 95% CI
average_model_pls_benefit = gpt4o_mini_evaluator_results["benefit_answer_mean_diff"].mean()
ci_lower_model_benefit, ci_upper_model_benefit = calculate_confidence_interval(gpt4o_mini_evaluator_results, "benefit_answer_mean_diff")

average_pls_model_rigor = gpt4o_mini_evaluator_results["rigor_answer_mean_diff"].mean()
ci_lower_model_rigor, ci_upper_model_rigor = calculate_confidence_interval(gpt4o_mini_evaluator_results, "rigor_answer_mean_diff")

average_pls_model_importance = gpt4o_mini_evaluator_results["importance_answer_mean_diff"].mean()
ci_lower_model_importance, ci_upper_model_importance = calculate_confidence_interval(gpt4o_mini_evaluator_results, "importance_answer_mean_diff")

average_pls_model_full_text = gpt4o_mini_evaluator_results["full_text_answer_mean_diff"].mean()
ci_lower_model_full_text, ci_upper_model_full_text = calculate_confidence_interval(gpt4o_mini_evaluator_results, "full_text_answer_mean_diff")

average_pls_model_another_trial = gpt4o_mini_evaluator_results["another_trial_answer_mean_diff"].mean()
ci_lower_model_another_trial, ci_upper_model_another_trial = calculate_confidence_interval(gpt4o_mini_evaluator_results, "another_trial_answer_mean_diff")

gpt4o_mini_pls_model_stats = {
    "benefit_answer": {"mean_diff": average_model_pls_benefit, "ci_lower": ci_lower_model_benefit, "ci_upper": ci_upper_model_benefit},
    "rigor_answer": {"mean_diff": average_pls_model_rigor, "ci_lower": ci_lower_model_rigor, "ci_upper": ci_upper_model_rigor},
    "importance_answer": {"mean_diff": average_pls_model_importance, "ci_lower": ci_lower_model_importance, "ci_upper": ci_upper_model_importance},
    "full_text_answer": {"mean_diff": average_pls_model_full_text, "ci_lower": ci_lower_model_full_text, "ci_upper": ci_upper_model_full_text},
    "another_trial_answer": {"mean_diff": average_pls_model_another_trial, "ci_lower": ci_lower_model_another_trial, "ci_upper": ci_upper_model_another_trial}
}

pls_gpt4o_mini_model_stats_df = pd.DataFrame(gpt4o_mini_pls_model_stats).T
pls_gpt4o_mini_model_stats_df["metric"] = pls_gpt4o_mini_model_stats_df.index
# remove index
pls_gpt4o_mini_model_stats_df.reset_index(drop=True, inplace=True)
pls_gpt4o_mini_model_stats_df["evaluator"] = "GPT4o Mini"

pls_gpt4o_mini_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,evaluator
0,3.592424,3.573721,3.611128,benefit_answer,GPT4o Mini
1,1.353030,1.332122,1.373939,rigor_answer,GPT4o Mini
2,2.730303,2.707594,2.753012,importance_answer,GPT4o Mini
3,3.686364,3.655170,3.717557,full_text_answer,GPT4o Mini
4,3.868182,3.840497,3.895866,another_trial_answer,GPT4o Mini


In [171]:
pls_claude_model_stats_df = calculate_model_stats(claude_evaluator_results, method_name = "Claude 3.5 Sonnet")

pls_claude_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,evaluator
0,2.471212,2.466320,2.476105,benefit_answer,Claude 3.5 Sonnet
1,-0.175758,-0.182107,-0.169408,rigor_answer,Claude 3.5 Sonnet
2,-0.337879,-0.345682,-0.330075,importance_answer,Claude 3.5 Sonnet
3,2.818182,2.804775,2.831589,full_text_answer,Claude 3.5 Sonnet
4,2.706061,2.687012,2.725109,another_trial_answer,Claude 3.5 Sonnet


In [172]:
# combine two dataframes
all_pls_model_stats_df = pd.concat([pls_gpt4o_mini_model_stats_df, pls_claude_model_stats_df], ignore_index=True)

all_pls_model_stats_df

,mean_diff,ci_lower,ci_upper,metric,evaluator
0,3.592424,3.573721,3.611128,benefit_answer,GPT4o Mini
1,1.353030,1.332122,1.373939,rigor_answer,GPT4o Mini
2,2.730303,2.707594,2.753012,importance_answer,GPT4o Mini
3,3.686364,3.655170,3.717557,full_text_answer,GPT4o Mini
4,3.868182,3.840497,3.895866,another_trial_answer,GPT4o Mini
5,2.471212,2.466320,2.476105,benefit_answer,Claude 3.5 Sonnet
6,-0.175758,-0.182107,-0.169408,rigor_answer,Claude 3.5 Sonnet
7,-0.337879,-0.345682,-0.330075,importance_answer,Claude 3.5 Sonnet
8,2.818182,2.804775,2.831589,full_text_answer,Claude 3.5 Sonnet
9,2.706061,2.687012,2.725109,another_trial_answer,Claude 3.5 Sonnet


In [173]:
# create altair grouped barchart
# grouped by metric and evaluator

# Create a mapping for custom facet titles
facet_title_mapping = {
    'benefit_answer': 'Benefit',
    'rigor_answer': 'Rigor',
    'importance_answer': 'Importance',
    'full_text_answer': 'Full-Text',
    'another_trial_answer': 'Another Trial'
}

# Define the desired order for the facets
facet_order = ['Benefit', 'Rigor', 'Importance', 'Full-Text', 'Another Trial']

color_mapping = {
    'Claude 3.5 Sonnet': '#0868ac',  
    'GPT4o Mini': '#43a2ca',  
}

method_order = ['Claude 3.5 Sonnet', 'GPT4o Mini']

# Apply the mapping as a calculated field
chart_data = all_pls_model_stats_df.copy()
chart_data['metric'] = chart_data['metric'].map(facet_title_mapping)

# Configure global font sizes
chart_config = {
    "axis": {"labelFontSize": 20, "titleFontSize": 22},  # Axis labels and titles
    "header": {"labelFontSize": 20, "titleFontSize": 22},  # Facet headers
    "legend": {"labelFontSize": 18, "titleFontSize": 20},  # Legend labels and titles
    "text": {"fontSize": 20},  # Text mark size
}

# Bar chart
bars = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('evaluator:N', title=None, axis=alt.Axis(labelAngle=-45), sort = method_order),
    y=alt.Y('mean_diff:Q', title='Mean Difference'),
    color=alt.Color('evaluator:N', title='Evaluator', legend=None, scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())))
).properties(
    width=120,  # Set the width to 300 pixels
    height=250  # Set the height to 300 pixels
)

# Error bars
error_bars = alt.Chart(chart_data).mark_errorbar().encode(
    alt.X("evaluator:N", sort = method_order),
    alt.Y("ci_lower:Q").title("Mean Difference"),
    alt.Y2("ci_upper:Q"),
    strokeWidth=alt.value(2),
    color=alt.value('gray')
)

# Add value labels
text = bars.mark_text(
    align='center',
    baseline='bottom',
    fontWeight='bold',
    dy=alt.expr(expr=alt.expr.if_(alt.datum.mean_diff >= 0, -1, 20))  # Adjust the position of the text    
).encode(
    text=alt.Text('mean_diff:Q', format='.2f'),
    color=alt.value('black')  # Set text color to black
)

# Combine layers and facet
chart = alt.layer(bars, text, error_bars, data=chart_data).facet(
    column=alt.Column('metric:N', title=None, sort=facet_order),
).configure(**chart_config)  # Apply

# save to html
chart.save("./plots/pls_evaluator_comparison_by_measures.html")

chart



alt.FacetChart(...)